# 优化

本笔记将介绍如何使用LangChain和[LangSmith](https://smith.langchain.com)来优化链。

## 设置

我们将设置一个环境变量用于LangSmith，并加载相关数据。

In [1]:
import os

# 设置环境变量 LANGCHAIN_PROJECT 为 "movie-qa"
os.environ["LANGCHAIN_PROJECT"] = "movie-qa"

In [2]:
import pandas as pd

In [17]:
# 导入 pandas 库
import pandas as pd

# 使用 pandas 的 read_csv 函数读取 "data/imdb_top_1000.csv" 文件，并将结果赋值给变量 df
df = pd.read_csv("data/imdb_top_1000.csv")

In [4]:
# 将 "Released_Year" 列的数据类型转换为整数类型
df["Released_Year"] = df["Released_Year"].astype(int, errors="ignore")

## 创建初始检索链

我们将使用自查询检索器

In [5]:
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [6]:
# 将DataFrame转换为字典列表
records = df.to_dict("records")

# 创建一个文档列表，每个文档对象包含页面内容和元数据
documents = [Document(page_content=d["Overview"], metadata=d) for d in records]

In [7]:
# 使用 Chroma 类的 from_documents 方法创建一个 vectorstore 对象
# 参数 documents 是一个文档列表，用于构建嵌入向量
# 参数 embeddings 是一个嵌入向量的数据集
vectorstore = Chroma.from_documents(documents, embeddings)

In [9]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="Released_Year",
        description="The year the movie was released",
        type="int",
    ),
    AttributeInfo(
        name="Series_Title",
        description="The title of the movie",
        type="str",
    ),
    AttributeInfo(
        name="Genre",
        description="The genre of the movie",
        type="string",
    ),
    AttributeInfo(
        name="IMDB_Rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"
llm = ChatOpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)

In [10]:
# 导入langchain_core包中的RunnablePassthrough类
from langchain_core.runnables import RunnablePassthrough

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [12]:
# 创建一个聊天提示模板，根据提供的信息回答用户的问题
prompt = ChatPromptTemplate.from_template(
    """Answer the user's question based on the below information:

Information:

{info}

Question: {question}"""
)

# 创建一个生成器，用于处理聊天提示并生成回答
generator = (prompt | ChatOpenAI() | StrOutputParser()).with_config(
    run_name="generator"
)

In [13]:
# 创建一个链式操作
chain = (
    # 使用RunnablePassthrough类的assign方法，将info字段赋值为question字段的值，并将结果传递给retriever函数
    RunnablePassthrough.assign(info=(lambda x: x["question"]) | retriever) | generator
)

## 运行示例

通过链式运行示例。可以手动运行，也可以使用示例列表或生产流量运行。

In [14]:
chain.invoke({"question": "what is a horror movie released in early 2000s"})

'One of the horror movies released in the early 2000s is "The Ring" (2002), directed by Gore Verbinski.'

## 注释

现在，转到LangSmitha并注释这些示例，标记为正确或不正确。

## 创建数据集

现在我们可以从这些运行中创建一个数据集。

我们要做的是找到标记为正确的运行，然后从中获取子链。具体来说，是查询生成器子链和最终生成步骤。

In [15]:
# 导入langsmith库中的Client类
from langsmith import Client

# 创建一个Client对象
client = Client()

In [16]:
# 调用client.list_runs()函数获取运行列表
# 参数project_name指定项目名称为"movie-qa"
# 参数execution_order指定执行顺序为1
# 参数filter指定过滤条件为"feedback_key为'correctness'且feedback_score为1"
runs = list(
    client.list_runs(
        project_name="movie-qa",
        execution_order=1,
        filter="and(eq(feedback_key, 'correctness'), eq(feedback_score, 1))",
    )
)

# 输出运行列表的长度
len(runs)

14

In [17]:
# 创建两个空列表，用于存储生成器和查询构造器的运行结果
gen_runs = []
query_runs = []

# 遍历runs列表中的每个元素
for r in runs:
    # 调用client.list_runs()方法获取生成器的运行结果，并将结果添加到gen_runs列表中
    gen_runs.extend(
        list(
            client.list_runs(
                project_name="movie-qa",
                filter="eq(name, 'generator')",
                trace_id=r.trace_id,
            )
        )
    )
    # 调用client.list_runs()方法获取查询构造器的运行结果，并将结果添加到query_runs列表中
    query_runs.extend(
        list(
            client.list_runs(
                project_name="movie-qa",
                filter="eq(name, 'query_constructor')",
                trace_id=r.trace_id,
            )
        )
    )

In [21]:
# 获取第一个运行实例的输入

runs[0].inputs

{'question': 'what is a high school comedy released in early 2000s'}

In [20]:
# 获取第一个运行实例的输出
runs[0].outputs

{'output': 'One high school comedy released in the early 2000s is "Mean Girls" starring Lindsay Lohan, Rachel McAdams, and Tina Fey.'}

In [22]:
# 获取query_runs列表中第一个元素的inputs属性
query_runs[0].inputs

{'query': 'what is a high school comedy released in early 2000s'}

In [23]:
# 查询运行列表的第一个元素的输出
query_runs[0].outputs

{'output': {'query': 'high school comedy',
  'filter': {'operator': 'and',
   'arguments': [{'comparator': 'eq', 'attribute': 'Genre', 'value': 'comedy'},
    {'operator': 'and',
     'arguments': [{'comparator': 'gte',
       'attribute': 'Released_Year',
       'value': 2000},
      {'comparator': 'lt', 'attribute': 'Released_Year', 'value': 2010}]}]}}}

In [24]:
# 获取gen_runs列表中第一个元素的inputs属性
gen_runs[0].inputs

{'question': 'what is a high school comedy released in early 2000s',
 'info': []}

In [25]:
# 获取 gen_runs 列表中第一个元素的 outputs 属性
gen_runs[0].outputs

{'output': 'One high school comedy released in the early 2000s is "Mean Girls" starring Lindsay Lohan, Rachel McAdams, and Tina Fey.'}

## 创建数据集

现在我们可以为查询生成和最终生成步骤创建数据集。
我们这样做是为了（1）可以检查数据点，（2）如果需要可以编辑它们，（3）随着时间的推移可以添加数据。

In [15]:
# 使用client创建名为"movie-query_constructor"的数据集
client.create_dataset("movie-query_constructor")

# 从query_runs中提取输入和输出数据
inputs = [r.inputs for r in query_runs]
outputs = [r.outputs for r in query_runs]

# 使用client创建示例，将提取的输入和输出数据作为参数，同时指定数据集名称为"movie-query_constructor"
client.create_examples(
    inputs=inputs, outputs=outputs, dataset_name="movie-query_constructor"
)

In [16]:
# 使用client对象创建名为"movie-generator"的数据集
client.create_dataset("movie-generator")

# 从gen_runs中提取输入和输出
inputs = [r.inputs for r in gen_runs]
outputs = [r.outputs for r in gen_runs]

# 使用client对象创建示例，将提取的输入和输出添加到名为"movie-generator"的数据集中
client.create_examples(inputs=inputs, outputs=outputs, dataset_name="movie-generator")

## 使用尽可能少的示例

我们现在可以下载一个数据集，并将它们用作未来链中的少量示例。

In [26]:
# 调用client对象的list_examples方法，传入参数dataset_name="movie-query_constructor"，获取示例列表
examples = list(client.list_examples(dataset_name="movie-query_constructor"))

In [27]:
import json


def filter_to_string(_filter):
    # 如果_filter字典中有"operator"键
    if "operator" in _filter:
        # 遍历_filter字典中的"arguments"键对应的值，并对每个值调用filter_to_string函数，将结果存入args列表中
        args = [filter_to_string(f) for f in _filter["arguments"]]
        # 返回一个格式化字符串，其中包含_filter字典中的"operator"键对应的值和args列表中的元素，元素之间用逗号分隔
        return f"{_filter['operator']}({','.join(args)})"
    else:
        # 将_filter字典中的"comparator"键对应的值赋给comparator变量
        comparator = _filter["comparator"]
        # 将_filter字典中的"attribute"键对应的值转换为JSON格式的字符串，并赋给attribute变量
        attribute = json.dumps(_filter["attribute"])
        # 将_filter字典中的"value"键对应的值转换为JSON格式的字符串，并赋给value变量
        value = json.dumps(_filter["value"])
        # 返回一个格式化字符串，其中包含comparator变量、attribute变量和value变量
        return f"{comparator}({attribute}, {value})"

In [28]:
# 创建一个空列表来存储模型示例
model_examples = []

# 遍历输入的示例列表
for e in examples:
    # 检查示例中是否包含"filter"字段
    if "filter" in e.outputs["output"]:
        # 如果包含"filter"字段，则将filter转换为字符串
        string_filter = filter_to_string(e.outputs["output"]["filter"])
    else:
        # 如果不包含"filter"字段，则将string_filter设置为"NO_FILTER"
        string_filter = "NO_FILTER"
    
    # 将查询和过滤器信息添加到模型示例列表中
    model_examples.append(
        (
            e.inputs["query"],  # 将输入示例中的查询信息添加到模型示例中
            {"query": e.outputs["output"]["query"], "filter": string_filter},  # 将输出示例中的查询和过滤器信息添加到模型示例中
        )
    )

In [29]:


# 创建 SelfQueryRetriever 实例 retriever1
# 使用 from_llm 方法从 llm 对象创建 retriever1
# 参数 llm: 指定的 llm 对象
# 参数 vectorstore: 指定的 vectorstore 对象
# 参数 document_content_description: 指定的 document_content_description 对象
# 参数 metadata_field_info: 指定的 metadata_field_info 对象
# 参数 verbose: 是否显示详细信息，默认为 True
# 参数 chain_kwargs: 一个字典，包含额外的参数，这里传入了一个 examples 参数，值为 model_examples
retriever1 = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    verbose=True,
    chain_kwargs={"examples": model_examples},
)

In [30]:
# 创建一个名为chain1的变量，它是一个函数链
chain1 = (
    # 第一个函数是RunnablePassthrough，它将输入作为输出传递
    RunnablePassthrough
    # 使用assign函数将一个名为info的键值对添加到输入中，值为输入字典中的"question"键对应的值
    .assign(info=(lambda x: x["question"]) | retriever1)
    # 将输入传递给generator函数
    | generator
)

In [31]:


# 调用chain1的invoke方法，并传入一个字典作为参数
# 字典中包含一个键值对，键为"question"，值为"what are good action movies made before 2000 but after 1997?"
chain1.invoke(
    {"question": "what are good action movies made before 2000 but after 1997?"}
)

'1. "Saving Private Ryan" (1998) - Directed by Steven Spielberg, this war film follows a group of soldiers during World War II as they search for a missing paratrooper.\n\n2. "The Matrix" (1999) - Directed by the Wachowskis, this science fiction action film follows a computer hacker who discovers the truth about the reality he lives in.\n\n3. "Lethal Weapon 4" (1998) - Directed by Richard Donner, this action-comedy film follows two mismatched detectives as they investigate a Chinese immigrant smuggling ring.\n\n4. "The Fifth Element" (1997) - Directed by Luc Besson, this science fiction action film follows a cab driver who must protect a mysterious woman who holds the key to saving the world.\n\n5. "The Rock" (1996) - Directed by Michael Bay, this action thriller follows a group of rogue military men who take over Alcatraz and threaten to launch missiles at San Francisco.'